# Prompt Engineering 入門

這個 Notebook 將帶您學習 **Prompt Engineering**，也就是如何設計提示（Prompts）來與 Azure OpenAI 模型互動，生成我們想要的回應。我們將展示以下內容：
- **一般用法**：簡單問答和總結
- **Few-shot 提示工程**：通過範例教模型分類
- **進階應用**：零次提示分類（Zero-shot Classification）和實體提取（Entity Extraction）

## 在 Anaconda 中設置虛擬環境並運行

我們將使用 Anaconda 的虛擬環境來運行這個 Notebook，確保依賴套件獨立於其他專案。請按照以下步驟設置環境：

### 步驟 1：創建虛擬環境
1. 開啟 **Anaconda Prompt**（Windows）或終端機（macOS/Linux）。
2. 創建一個新的虛擬環境，命名為 `prompt_env`，並指定 Python 版本（例如 3.11.7）：
   ```bash
   conda create -n prompt_env python=3.11.7
   ```
3. 啟動虛擬環境：
   ```bash
   conda activate prompt_env
   ```
   啟動後，命令提示符前會顯示 `(prompt_env)`。

### 步驟 2：安裝所需套件
在啟動的虛擬環境中，安裝以下套件：
- `openai`：用於與 Azure OpenAI API 互動。
- `jupyter`：確保 Jupyter Notebook 可用（Anaconda 通常已預裝）。

執行以下命令：
```bash
conda install jupyter
pip install openai python-dotenv
```

### 步驟 3：啟動 Jupyter Notebook
1. 在虛擬環境中，運行以下命令啟動 Jupyter Notebook：
   ```bash
   jupyter notebook
   ```
2. 在瀏覽器中打開Jupyter Notebook
3. 在瀏覽器中打開這個 `prompt_engineering.ipynb` 檔案。

確保這個檔案與 Notebook 在同一目錄下，程式碼會自動載入這些變數。

### 注意事項
- 如果 Jupyter Notebook 未正確使用虛擬環境的內核，請在 Notebook 介面中點擊「Kernel」 > 「Change Kernel」，選擇 `prompt_env`。
- 若內核未顯示，運行以下命令註冊內核：
   ```bash
   python -m ipykernel install --user --name prompt_env --display-name "prompt_env"
   ```

In [ ]:
aoai_key = "請輸入AOAI API Key"
aoai_url = "請輸入AOAI API Endpoint"
aoai_model_version = "請輸入 AOAI model"
api_version= "請輸入 AOAI API Version"

In [ ]:
# 導入必要的套件
import os
from openai import AzureOpenAI

def chat_with_aoai_gpt(messages, temperature=0.7):
    """與 Azure OpenAI 模型對話的核心函數"""
    error_time = 0
    while error_time <= 2:
        error_time += 1
        try:        
            client = AzureOpenAI(
                api_key=aoai_key,
                azure_endpoint=aoai_url,
                api_version=api_version,
            )
            response = client.chat.completions.create(
                model=aoai_model_version,
                messages=messages,
                temperature=temperature,
            )

            assistant_message = response.choices[0].message.content
            return (assistant_message, response.usage.prompt_tokens, response.usage.total_tokens - response.usage.prompt_tokens)
        except Exception as e:
            print(f"錯誤：{str(e)}")
            return "", 0, 0

## 1. 一般用法

這裡展示如何使用簡單的提示來獲得答案或總結內容。Prompt Engineering 的核心是設計清晰的輸入，讓模型理解我們的意圖。

In [ ]:
# 定義一般用法函數
def get_response(user_message):
    messages = [
        {"role": "system", "content": "您是一個樂於助人的助理。"},
        {"role": "user", "content": user_message}
    ]
    return chat_with_aoai_gpt(messages)

In [ ]:
# 示例 1：描述三個關於猴子的事實
response = get_response("描述三個關於猴子的事實")
print(f"三個關於猴子的事實:\n{response[0]}\n")

In [ ]:
# 示例 2：草是什麼顏色？
response = get_response("草是什麼顏色？用中文回答")
print(f"草是什麼顏色:\n{response[0]}\n")

In [ ]:
# 示例 3：總結司馬光的故事
story = """總結以下的故事：\n司馬光生於光州光山，有一次，他跟幾個小夥伴在後院玩耍。有一個孩子淘氣，他爬到一口大水缸上，結果失足掉進去了。水缸深，孩子小，眼看小夥伴就要淹死了，其他的孩子都嚇傻了，有的孩子嚇得大哭，有的孩子嚇得去找大人。就在此時，司馬光急中生智，從地上撿起一塊大石頭，使勁向水缸擊去。通過司馬光的砸缸行為，水湧出來，小夥伴因此得救了。"""
response = get_response(story)
print(f"故事總結:\n{response[0]}\n")

## 2. Few-shot 提示工程

Few-shot 提示通過提供少數範例，讓模型學會執行特定任務，例如情感分類。這裡我們用推文的情感（積極或消極）作為例子。

In [ ]:
# 定義 Few-shot 函數
def get_few_shot_response(user_message):
    messages = [
        {"role": "system", "content": """twitter 是一個社交媒體平台，用戶可以發佈推文。推文可以是積極的或消極的，我們希望能夠將推文分類成積極或消極。以下是一些積極和消極推文的例子。請確保正確分類最後一個推文是積極的還是消極的"""},
        {"role": "user", "content": "今天真是開心的一天"},
        {"role": "assistant", "content": "積極的"},
        {"role": "user", "content": "我討厭這個班級"},
        {"role": "assistant", "content": "消極的"},
        {"role": "user", "content": "我喜歡這杯咖啡"},
        {"role": "assistant", "content": "積極的"},
        {"role": "user", "content": user_message}
    ]
    return chat_with_aoai_gpt(messages)

In [ ]:
# 测试 Few-shot 情感分類
response = get_few_shot_response("今天工作進度都滿順利的")
print(f"推文語意判斷:\n{response[0]}\n")

## 3. 進階應用：Zero-shot Classification

Zero-shot 分類不需要範例，直接讓模型根據描述進行分類。這裡我們分析一段消費者投訴，判斷它屬於哪個類別。

In [ ]:
# Zero-shot 分類
complaint = """以下段落是一則消費者投訴。投訴內容涉及以下選項之一：信用卡、信用報告、抵押貸款與貸款、零售銀行業務或債務追討。請閱讀以下段落並判斷投訴屬於哪個選項。
我多年來一直在富國銀行（Wells Fargo）持有抵押貸款。每個月我都會提前7-10天付款。在 XX/XX/XXXX 至 XX/XX/XXXX 期間，我每月支付 $3000.00。在 XXXX 年，我接到富國銀行的電話，說我的月付款金額不正確。經過長時間討論，我同意額外支付 $750.00 以使帳戶恢復正常，並從此支付 $XXXX。在 XX/XX/XXXX，我收到一封來自 XXXX 的信，稱我的抵押貸款已違約，並建議我立即採取行動。經過長時間討論，我終於發現，在 XX/XX/XXXX，銀行如常收到我的付款，但因為金額低於他們的要求，他們沒有將這筆錢用於支付我的抵押貸款，而是將全部金額應用於本金。他們從未通知我。他們一直向信用機構報告我，還威脅要沒收我的房子，聲稱我未付款，而事實上我從未漏付或遲交。他們這樣對待我，卻連通知都沒有。為什麼他們不打電話給我？他們檔案中有兩個電話號碼，其中一個已經停用20年，他們從未撥打另一個號碼。我注意到我在 XXXX 年與一位年輕人通話時，他能通過電話聯繫到我。為什麼不寄信？他們為什麼這樣對我？他們說是電腦造成的。為什麼他們不能回溯修復？他們說時間太久了。我必須在今年支付第13次抵押貸款，否則將面臨沒收。他們在欺騙我。你會以為可以信任銀行管理你的帳戶，但現在我明白這不是真的。我已經 XXXX 歲了，也許這就是他們採取這種政策的理由。"""
response = get_response(complaint)
print(f"投訴類別:\\n{response[0]}\\n")

## 4. 進階應用：One-shot Entity Extraction

One-shot 實體提取通過一個範例教模型提取個人身份資訊（PII）。這裡我們提供一個範例，然後測試新輸入。

In [ ]:
# One-shot 實體提取
def get_one_shot_entity_extraction(user_message):
    messages = [
        {"role": "system", "content": "從給定的段落中提取個人身份資訊（PII）實體。"},
        {"role": "user", "content": "我從紐約的銀行通過手機提領了 $100。電話號碼 (345) 123-7867。問候，Raj"},
        {"role": "assistant", "content": "1. 金額: $100\\n2. 地點: 紐約\\n3. 電話號碼: (345) 123-7867\\n4. 姓名: Raj"},
        {"role": "user", "content": user_message}
    ]
    return chat_with_aoai_gpt(messages)

# 測試一次提示實體提取
test_input = "嗨，我是 Ravi Dube。我在 2023 年 3 月 30 日的信用卡對帳單上注意到一筆 $1,000 的費用。該交易是在紐約的一家餐廳進行的。請通過 (123)456-7890 或 ravi.dube@email.com 聯繫我。"
response = get_one_shot_entity_extraction(test_input)
print(f"提取的實體:\\n{response[0]}\\n")

## 5. 進階應用：Two-shot Entity Extraction

Two-shot 實體提取提供兩個範例，讓模型更準確地提取 PII。這裡我們測試相同的輸入，看看是否更精確。

In [ ]:
# Two-shot 實體提取
# 二次提示實體提取
def get_two_shot_entity_extraction(user_message):
    messages = [
        {"role": "system", "content": "從給定的段落中提取個人身份資訊（PII）實體。"},
        {"role": "user", "content": "我從紐約的銀行通過手機提領了 $100。電話號碼 (345) 123-7867。問候，Raj"},
        {"role": "assistant", "content": "1. 金額: $100\\n2. 地點: 紐約\\n3. 電話號碼: (345) 123-7867\\n4. 姓名: Raj"},
        {"role": "user", "content": "感謝您聯繫我。我的信用卡在海外度假時被取消。我的信用卡號是 39482374859，帳戶名稱是 Phil Smith，地址是 34 Cityvale, Melbourne, 3000。我偏好的聯繫方式是電子郵件：phil.smith@email.com。"},
        {"role": "assistant", "content": "1. 信用卡號: 39482374859\\n2. 姓名: Phil Smith\\n3. 地點: 34 Cityvale, Melbourne, 3000\\n4. 電子郵件地址: phil.smith@email.com"},
        {"role": "user", "content": user_message}
    ]
    return chat_with_aoai_gpt(messages)

# 測試二次提示實體提取
test_input = "嗨，我是 Ravi Dube。我在 2023 年 3 月 30 日的信用卡對帳單上注意到一筆 $1,000 的費用。該交易是在紐約的一家餐廳進行的。請通過 (123)456-7890 或 ravi.dube@email.com 聯繫我。"
response = get_two_shot_entity_extraction(test_input)
print(f"提取的實體:\\n{response[0]}\\n")

## 總結

通過這個 Notebook，您已經學習了：
- 如何在 Anaconda 中設置虛擬環境並安裝套件。
- 如何使用簡單提示進行問答和總結。
- 如何通過 Few-shot 提示教模型分類情感。
- 如何使用 Zero-shot 和 One/Two-shot 提示進行分類和實體提取。

建議您嘗試修改提示，觀察模型的回應變化，並探索更多 Prompt Engineering 技巧！